In [ ]:
import os
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# matplotlib.use("agg")

from exp_spec_info import *
from plot_info import *
from select_data import *

In [ ]:
# Processed median path
median_data_path = "C:\\Users\\dosre\\dev\\thesis-data\\median_data.pkl"

# Plot output directory
plot_output_root = "C:\\Users\\dosre\\dev\\thesis-data\\plots"
os.makedirs(plot_output_root, exist_ok=True)
pc_gmres_m_64_comp_plots_dir = os.path.join(plot_output_root, "pc_gmres_m_64")
os.makedirs(pc_gmres_m_64_comp_plots_dir, exist_ok=True)
pc_gmres_m_32_comp_plots_dir = os.path.join(plot_output_root, "pc_gmres_m_32")
os.makedirs(pc_gmres_m_32_comp_plots_dir, exist_ok=True)

In [ ]:
# Load data and filter for only gmres m solvers
median_data = pd.read_pickle(median_data_path)
median_data = median_data[median_data["solver"].transform(lambda x: (x in PC_SOLVERS))]

##### 7.3/7.4 FP-GMRES and PC-GMRES Comparison

In [ ]:
for setup, data_point, data_label, fp64_val, x_range, save_dir in [
    ("unprecond", "med_rel_time_fp64", "Rel. Time. to FP64", 1., [0., 3.], pc_gmres_m_64_comp_plots_dir),
    ("unpreconddense", "med_rel_time_fp64", "Rel. Time. to FP64", 1., [0., 3.], pc_gmres_m_64_comp_plots_dir),
    ("unprecond", "med_rel_res_frac_err_fp64", "Rel. Res. Frac. Error to FP64", 0., [-2., 5.], pc_gmres_m_64_comp_plots_dir),
    ("unpreconddense", "med_rel_res_frac_err_fp64", "Rel. Res. Frac. Error to FP64", 0., [-2., 5.], pc_gmres_m_64_comp_plots_dir),
    ("unprecond", "med_rel_time_fp32", "Rel. Time. to FP32", 1., [0., 3.], pc_gmres_m_32_comp_plots_dir),
    ("unpreconddense", "med_rel_time_fp32", "Rel. Time. to FP32", 1., [0., 3.], pc_gmres_m_32_comp_plots_dir),
    ("unprecond", "med_rel_res_frac_err_fp32", "Rel. Res. Frac. Error to FP32", 0., [-2., 5.], pc_gmres_m_32_comp_plots_dir),
    ("unpreconddense", "med_rel_res_frac_err_fp32", "Rel. Res. Frac. Error to FP32", 0., [-2., 5.], pc_gmres_m_32_comp_plots_dir),
]:

    setup_data = median_data[median_data["setup"] == setup]

    for prefix, solvers in [
        ("SD", ["PC SD ORC", "PC SD CS", "PC SD RRT"]),
        ("HSD", ["PC HSD ORC", "PC HSD CS", "PC HSD RRT", "PC HSD S2T"])
    ]:

        fig, axs = plt.subplots(
            1, len(solvers), figsize=(len(solvers)*8.5/3, 8.2), sharey=True
        )

        for ax, solver in zip(axs, solvers):

            solver_data = setup_data[setup_data["solver"] == solver]
            ax.scatter(
                solver_data[data_point],
                solver_data["matrix"],
                alpha=1.,
                color=SOLVER_CLR_DICT[solver],
                zorder=2,
                marker="."
            )
            ax.axvline(
                fp64_val,
                color=SOLVER_CLR_DICT["FP FP64"],
                linestyle="--",
                zorder=1
            )
            ax.axvline(
                solver_data[data_point].median(),
                color=SOLVER_CLR_DICT[solver],
                linestyle=":",
                zorder=2
            )
            ax.set_title(f"{solver}")

            ax.set_xlim(x_range[0], x_range[1])
            off_screen_data = solver_data[solver_data[data_point] >= x_range[1]]["matrix"].value_counts()
            for matrix in off_screen_data.index:
                ax.text(
                    x_range[0]+0.875*(x_range[1]-x_range[0]), matrix,
                    f"{off_screen_data.loc[matrix]}$\\rightarrow$",
                    verticalalignment='center'
                )

        for ax in axs:
            ax.set_xlabel(f"Med. {data_label}")
            ax.grid()
        axs[0].set_yticks(solver_data["matrix"].unique())
        axs[0].set_ylabel("Experiment")

        fig.suptitle(f"{SETUP_NAME_MAPPING[setup]} {prefix} PC-GMRES($m$) Solver {data_label} Performance")
        fig.tight_layout()

        data_tag = "-".join(data_label.replace(".", "").split(" ")).lower()
        plt.savefig(
            os.path.join(save_dir, f"{data_tag}-{setup}-{prefix.lower()}.png")
        )
        plt.show()
        plt.close()


##### Misc Analysis

In [ ]:
small_matrix_data_path = "C:\\Users\\dosre\\dev\\precision-cascade\\scripts\\python\\view_experimentation\\data\\small-experimental-matrix-data.csv"
large_matrix_data_path = "C:\\Users\\dosre\\dev\\precision-cascade\\scripts\\python\\view_experimentation\\data\\large-experimental-matrix-data.csv"

In [ ]:
small_matrix_data = pd.read_csv(small_matrix_data_path).rename({"name": "matrix"}, axis=1)
large_matrix_data = pd.read_csv(large_matrix_data_path).rename({"name": "matrix"}, axis=1)

In [ ]:
solver_data = median_data[
    (median_data["setup"] == "unprecond")
]

small_merge = solver_data.merge(small_matrix_data, on="matrix", how="inner")
large_merge = solver_data.merge(large_matrix_data, on="matrix", how="inner")

fig, ax = plt.subplots()
ax.scatter(small_merge["med_inner_iter"], small_merge["med_rel_time_fp64"], marker=".")
ax.scatter(large_merge["med_inner_iter"], large_merge["med_rel_time_fp64"], marker=".")
# ax.semilogx()
# ax.set_yscale("symlog")
print(np.corrcoef(
    np.hstack([small_merge["med_inner_iter"], large_merge["med_inner_iter"]]),
    np.hstack([small_merge["med_rel_time_fp64"], large_merge["med_rel_time_fp64"]])
))
plt.show()

fig, ax = plt.subplots()
ax.scatter(small_merge["nnz"], small_merge["med_rel_time_fp64"], marker=".")
ax.scatter(large_merge["nnz"], large_merge["med_rel_time_fp64"], marker=".")
# ax.semilogx()
# ax.set_yscale("symlog")
print(np.corrcoef(
    np.hstack([small_merge["nnz"], large_merge["nnz"]]),
    np.hstack([small_merge["med_rel_time_fp64"], large_merge["med_rel_time_fp64"]])
))
plt.show()

fig, ax = plt.subplots()
ax.scatter(small_merge["rows"], small_merge["med_rel_time_fp64"], marker=".")
ax.scatter(large_merge["rows"], large_merge["med_rel_time_fp64"], marker=".")
# ax.semilogx()
# ax.set_yscale("symlog")
print(np.corrcoef(
    np.hstack([small_merge["rows"], large_merge["rows"]]),
    np.hstack([small_merge["med_rel_time_fp64"], large_merge["med_rel_time_fp64"]])
))
plt.show()

fig, ax = plt.subplots()
ax.scatter(small_merge["cond"], small_merge["med_rel_time_fp64"], marker=".")
# ax.semilogx()
# ax.set_yscale("symlog")
print(np.corrcoef(small_merge["cond"], small_merge["med_rel_time_fp64"]))
plt.show()